# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [59]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dicabisagan,122.4157,17.0818,28.24,73,81,1.98,PH,1670118687
1,1,atuona,-139.0333,-9.8000,25.37,78,2,6.65,PF,1670118837
2,2,thompson,-97.8558,55.7435,-27.91,76,20,2.06,CA,1670118838
3,3,saldanha,17.9442,-33.0117,14.03,75,53,8.72,ZA,1670118838
4,4,san+cristobal,-72.2250,7.7669,20.10,97,92,1.46,VE,1670118839


In [61]:
%%capture --no-display
# Configure the map plot

# Build the endpoint URL
target_city = []
for city in city_data_df['City']:
    target_city.append(city)
    
    url = 'https://api.geoapify.com/v1/icon?'
    target_url = f"{url}text={city}&format=json&apiKey={geoapify_key}"

# Display the map
city_data_df.hvplot.points("Lat","Lng", global_extent=True, geo=True, alpha = 0.5, 
                           scale = 1, tiles=True,  frame_width = 600,
                           frame_height = 450, size="Humidity", color="City")

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lat,Lng]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [33]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Max Temp'] >= 21 ) & (city_data_df['Max Temp'] < 27)
                            & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] < 65)]

# Drop any rows with null values
ideal_df = ideal_df.dropna()

# Display sample data
ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,salalah,54.0924,17.0151,21.90,57,0,1.44,OM,1670118845
52,52,bengkulu,102.2655,-3.8004,26.78,74,44,1.14,ID,1670118862
71,71,taoudenni,-3.9836,22.6783,21.38,19,0,2.73,ML,1670118872
75,75,ormara,64.6357,25.2088,22.94,47,0,2.00,PK,1670118874
78,78,cabo+san+lucas,-109.9124,22.8909,26.40,72,4,3.22,MX,1670118780


### Step 3: Create a new DataFrame called `hotel_df`.

In [34]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
18,salalah,OM,54.0924,17.0151,57,
52,bengkulu,ID,102.2655,-3.8004,74,
71,taoudenni,ML,-3.9836,22.6783,19,
75,ormara,PK,64.6357,25.2088,47,
78,cabo+san+lucas,MX,-109.9124,22.8909,72,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [35]:
# Set parameters to search for a hotel
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude}, {latitude}, {radius}"
    params["bias"] = f"proximity:{longitude}, {latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionaty
    name_address = requests.get(f'https://api.geoapify.com/v2/places?categories=accommodation.hotel\
                                &filter=circle:{latitude},{longitude},10000&apiKey={geoapify_key}')
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
# Display sample data
hotel_df


Starting hotel search
salalah - nearest hotel: Hilton Salalah Resort
bengkulu - nearest hotel: Hotel Andalas Permai
taoudenni - nearest hotel: No hotel found
ormara - nearest hotel: No hotel found
cabo+san+lucas - nearest hotel: Waldorf Astoria Los Cabos Pedregal
yaan - nearest hotel: No hotel found
mudgee - nearest hotel: Parklands Resort & Conference Centre
mahebourg - nearest hotel: Preskil Island Resort
kapaa - nearest hotel: Sheraton Kauai Coconut Beach Resort
chirongui - nearest hotel: Hôtel Le Jardin Maoré
champerico - nearest hotel: Posada Del Mar
kaeo - nearest hotel: No hotel found
grand+gaube - nearest hotel: Hibiscus
tsaratanana - nearest hotel: No hotel found
clearwater - nearest hotel: The Beachview Inn Clearwater Beach
road+town - nearest hotel: Sebastian's on the Beach
souillac - nearest hotel: Hotel SkylineInn
sur - nearest hotel: Resort Sur Beach Hotel
bambous+virieux - nearest hotel: Preskil Island Resort
port+macquarie - nearest hotel: ibis Styles Port Macquarie
bei

,City,Country,Lat,Lng,Humidity,Hotel Name
18,salalah,OM,54.0924,17.0151,57,Hilton Salalah Resort
52,bengkulu,ID,102.2655,-3.8004,74,Hotel Andalas Permai
71,taoudenni,ML,-3.9836,22.6783,19,No hotel found
75,ormara,PK,64.6357,25.2088,47,No hotel found
78,cabo+san+lucas,MX,-109.9124,22.8909,72,Waldorf Astoria Los Cabos Pedregal
...,...,...,...,...,...,...
583,nishihara,JP,127.7558,26.1842,78,チャビラホテル那覇
589,bama,NG,13.6856,11.5221,34,No hotel found
594,brazzaville,CG,15.2832,-4.2658,94,Elbo Suites
595,ailigandi,PA,-78.0167,9.2333,83,Akwadup Lodge


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [70]:
%%capture --no-display

# Configure the map plot
hotel_df.hvplot.points('Lat', 'Lng', global_extent=True, geo=True, alpha = 0.5, scale = 1, tiles=True, 
                           frame_width = 600, frame_height = 500, size="Humidity", color="City", hover_cols='all')
# Display the map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lat,Lng]   (City,Humidity,index,Country,Hotel Name)